In [10]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import psycopg2

def sliding_windows(text, tokenizer, window_size=512, stride=256):
    """
    Génère des sous-textes à partir de 'text' en utilisant une fenêtre
    glissante de taille 'window_size', chevauchant de 'stride'.
    Le dernier bloc sera ajusté à 'window_size' si le texte est suffisamment long.
    Renvoie (chunk_text, start_token_idx, end_token_idx).
    """
    tokens = tokenizer.encode(text, add_special_tokens=False)
    n_tokens = len(tokens)

    # Si le texte est inférieur à la taille de la fenêtre, ne pas diviser
    if n_tokens <= window_size:
        yield text, 0, n_tokens
        return

    i = 0
    while i < n_tokens:
        # Si le prochain saut dépasse la fin, on ajuste le début du bloc final
        if i + window_size >= n_tokens:
            i = max(0, n_tokens - window_size)

        # On prend un morceau de 'tokens' depuis i jusqu'à i+window_size
        chunk_tokens = tokens[i : i + window_size]
        if not chunk_tokens:
            break

        # Redécoder pour récupérer la string
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)

        # Indices de token effectifs
        start_idx = i
        end_idx = i + len(chunk_tokens)

        yield chunk_text, start_idx, end_idx

        # Arrêter si on est à la fin
        if i + window_size >= n_tokens:
            break

        # Avancer de 'stride' (et non pas de window_size)
        i += stride

def analyze_with_sliding_window(text, sentiment_pipeline, tokenizer, window_size=512, stride=256):
    """
    Applique un pipeline d'analyse de sentiments sur des fenêtres glissantes.

    Retourne une liste de dicts :
    [
        {
            "start_token_idx": <int>,
            "end_token_idx": <int>,
            "sentiment": <résultat du pipeline (liste complète)>
        },
        ...
    ]
    """
    results = []
    for chunk_text, start_idx, end_idx in sliding_windows(text, tokenizer, window_size, stride):
        # Le pipeline renvoie une liste (ex: [{'label': '4 stars', 'score': 0.997}, ...])
        # Ici, on récupère directement TOUT le résultat sans se limiter au premier élément
        sentiment_result = sentiment_pipeline(chunk_text)

        results.append({
            "start_token_idx": start_idx,
            "end_token_idx": end_idx,
            "sentiment": sentiment_result  # on stocke la liste entière
        })
    return results

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import psycopg2
from tqdm import tqdm

def process_and_store_sentiments(
    db_host="localhost",
    db_port=5432,
    db_name="postgres",
    db_user="jeremie",
    db_password="",
    table_transcript="transcript",
    table_sentiment="sentiment",
    model_name="nlptown/bert-base-multilingual-uncased-sentiment",
    window_size=512,
    stride=256
):
    """
    Exemple de fonction qui :
    1) se connecte à PostgreSQL
    2) lit la table 'transcript' (video_id, content)
    3) applique l'analyse sliding window
    4) stocke dans la table 'sentiment'
    """

    # -- Charger le modèle et le tokenizer --
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # -- Créer la pipeline --
    sentiment_pipeline = pipeline(
        'sentiment-analysis',
        model=model,
        tokenizer=tokenizer,
        truncation=True,  # Sécurité supplémentaire
        max_length=512
    )

    # -- Connexion à la DB (psycopg2) --
    conn_string = f"host={db_host} port={db_port} dbname={db_name} user={db_user} password={db_password}"
    with psycopg2.connect(conn_string) as conn:
        with conn.cursor() as cur:
            # 1) Récupérer les 20 premières transcriptions seulement (grâce à LIMIT 20)
            select_query = f"""
                SELECT transcript_video_id, transcript_content
                FROM {table_transcript}
            """
            cur.execute(select_query)
            rows = cur.fetchall()

            # 2) Pour chaque transcription (avec barre de progression)
            for row in tqdm(rows, desc="Processing transcripts"):
                video_id, content = row

                # Appliquer l'analyse en fenêtre glissante
                results = analyze_with_sliding_window(
                    text=content,
                    sentiment_pipeline=sentiment_pipeline,
                    tokenizer=tokenizer,
                    window_size=window_size,
                    stride=stride
                )

                # 3) Insérer chaque chunk dans la table "sentiment"
                insert_query = f"""
                    INSERT INTO {table_sentiment}
                    (sentiment_video_id,
                     sentiment_start_token_idx,
                     sentiment_end_token_idx,
                     sentiment_model_name,
                     sentiment)
                    VALUES (%s, %s, %s, %s, %s)
                    ON CONFLICT (sentiment_video_id, sentiment_start_token_idx, sentiment_end_token_idx, sentiment_model_name)
                    DO NOTHING
                """

                for res in results:
                    start_idx = res["start_token_idx"]
                    end_idx = res["end_token_idx"]

                    # sentiment_info est une liste
                    sentiment_info = res["sentiment"]
                    # Convertir en JSON (string) pour l’insérer en base
                    sentiment_json = json.dumps(sentiment_info)

                    cur.execute(insert_query, (
                        video_id,
                        start_idx,
                        end_idx,
                        model_name,
                        sentiment_json
                    ))

            # -- Fin de la boucle, on commit pour valider les insertions --
            conn.commit()

In [21]:
import json

In [22]:
process_and_store_sentiments(
    db_host="localhost",
    db_port=5432,
    db_name="postgres",
    db_user="jeremie",
    db_password="",
    table_transcript="transcript",
    table_sentiment="sentiment",
    model_name="nlptown/bert-base-multilingual-uncased-sentiment",
    window_size=512,
    stride=256
)

Device set to use mps:0
Processing transcripts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38473/38473 [12:53:54<00:00,  1.21s/it]
